# Cosine Similarity-Based Product Recommendation
This workflow demonstrates how to build a feature matrix and use cosine similarity to recommend similar products.

In [1]:
# Import libraries
import numpy as np
import sys
import os
import pandas as pd
from sklearn.metrics import ndcg_score
sys.path.append(os.path.abspath('../src'))
from model_training import get_recommendations, get_content_recommendations_pca, get_sentiment_recommendations, get_recommendations_with_pca, get_review_recommendations, get_topic_recommendations, get_reviewer_overlap_recommendations, get_weighted_hybrid_recommendations, load_all_models, get_content_recommendations, svd_product_recommendations, get_knn_recommendations, prepare_features_for_knn
# Load the DataFrame and set it in the ml module
from data_preprocessing import main
#main()
#df = pd.read_parquet('../data/processed/amazon_clean.parquet').reset_index(drop=True)
df = pd.read_parquet('../data/processed/amazon_clean.parquet')
#print(df.head())
load_all_models()

Recommendation function 'get_recommendations' updated to use reduced feature set.
All models and data loaded successfully.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.5.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 1.5.1 when using version 1.8.0. This might lead to breaking

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,379.0,₹919,59%,4.0,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1347,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2280.0,"₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but 

# Collaborative Filtering: Matrix Factorization (SVD)
This cell demonstrates how to use Singular Value Decomposition (SVD) for collaborative filtering product recommendations. SVD is a popular matrix factorization technique for recommender systems.

In [2]:
import os
sys.path.append(os.path.abspath('../src'))
from model_training import svd_product_recommendations

query_product_id = "B07JW9H4J1"  # Replace with your desired product_id
try:
    similar_products = svd_product_recommendations(df, query_product_id, n=5)
    print(similar_products)
except Exception as e:
    print(f"Error running svd_product_recommendations: {e}")

     product_id                                       product_name  rating  \
0    B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...     4.2   
42   B07JW1Y6XV  Wayona Nylon Braided 3A Lightning to USB A Syn...     4.2   
80   B07LGT55SJ  Wayona Usb Nylon Braided Data Sync And Chargin...     4.2   
106  B07JGDB5M1  Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...     4.2   
220  B07JH1CBGW  Wayona Nylon Braided Usb Syncing And Charging ...     4.2   

     discounted_price  
0               399.0  
42              399.0  
80              399.0  
106             449.0  
220             649.0  


In [3]:
from model_training import svd_product_recommendations

# Example: Evaluate SVD-based recommendations for a specific product_id
query_product_id = "B07JW9H4J1"  # Replace with your desired product_id
try:
    similar_products_svd = svd_product_recommendations(df, query_product_id, n=5)
    print(f"Top similar products for product_id {query_product_id} (SVD):")
    print(similar_products_svd)
except Exception as e:
    print(f"Error: {e}")

Top similar products for product_id B07JW9H4J1 (SVD):
     product_id                                       product_name  rating  \
0    B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...     4.2   
42   B07JW1Y6XV  Wayona Nylon Braided 3A Lightning to USB A Syn...     4.2   
80   B07LGT55SJ  Wayona Usb Nylon Braided Data Sync And Chargin...     4.2   
89   B07JH1C41D  Wayona Nylon Braided (2 Pack) Lightning Fast U...     4.2   
106  B07JGDB5M1  Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...     4.2   

     discounted_price  
0               399.0  
42              399.0  
80              399.0  
89              649.0  
106             449.0  


# Content-Based Filtering: KNN on Product Features
This cell demonstrates how to use K-Nearest Neighbors (KNN) on product features for content-based recommendations.

In [4]:
import os
sys.path.append(os.path.abspath('../src'))
from model_training import prepare_features_for_knn, get_knn_recommendations

# Prepare features
df, X = prepare_features_for_knn(df)

# Example: Query for a specific product_id
query_product_id = "B07JW9H4J1"  # Replace with your desired product_id

# Find the row index for this product_id
if query_product_id in df['product_id'].values:
    product_idx = int(df.index[df['product_id'] == query_product_id][0])
    try:
        similar_products = get_knn_recommendations(df, X, product_idx, 5)
        display(similar_products)
    except Exception as e:
        print(f"KNN error: {e}")
else:
    print(f"Product ID {query_product_id} not found in the DataFrame.")

,product_id,product_name,rating,discounted_price
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,4.2,399.0
80,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,4.2,399.0
166,B07JPJJZ2H,Wayona Nylon Braided Lightning USB Data Sync &...,4.2,399.0
820,B08MTLLSL8,boAt Bassheads 102 Wired in Ear Earphones with...,4.2,399.0
1132,B0814LP6S9,PrettyKrafts Laundry Basket for clothes with L...,3.7,351.0


## Cosine Similarity (Classic)
This method uses cosine similarity on product features to recommend similar products.

In [5]:
import os
sys.path.append(os.path.abspath('../src'))
from model_training import get_recommendations

product_id = "B07JW9H4J1"

try:
    pca_recs = get_recommendations(df, product_id, N=5)
    if isinstance(pca_recs, pd.DataFrame) and not pca_recs.empty:
        display(pca_recs)
    else:
        print("No recommendations found or Product ID not in database.")
except Exception as e:
    print(f"Error generating recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...
1,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,[High Compatibility] : Compatible For iPhone X...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-Syncing-C...
2,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
3,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,449.0,0.0,0.0,4.2,0.0,"[High Compatibility] : Phone X/XsMax/Xr ,Phone...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN6LG1-Sy...


## Cosine Similarity with PCA
This method applies PCA to reduce feature dimensionality before computing cosine similarity.

In [6]:
import os
sys.path.append(os.path.abspath('../src'))
from model_training import get_recommendations_with_pca

product_id = "B07JW9H4J1"

try:
    pca_recs = get_recommendations_with_pca(df, product_id, N=5)
    if isinstance(pca_recs, pd.DataFrame) and not pca_recs.empty:
        display(pca_recs)
    else:
        print("No recommendations found or Product ID not in database.")
except Exception as e:
    print(f"Error generating recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...
1,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,[High Compatibility] : Compatible For iPhone X...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-Syncing-C...
2,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
3,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,449.0,0.0,0.0,4.2,0.0,"[High Compatibility] : Phone X/XsMax/Xr ,Phone...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN6LG1-Sy...


## Review-Based Recommendations (TF-IDF)
This method uses TF-IDF vectorization of product reviews to recommend similar products.

In [7]:
from model_training import get_review_recommendations

try:
    review_recs = get_review_recommendations(df, product_id)
    if isinstance(review_recs, pd.DataFrame) and not review_recs.empty:
        display(review_recs)
    else:
        print("No review-based recommendations found.")
except Exception as e:
    print(f"Error generating review recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...
2,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,449.0,0.0,0.0,4.2,0.0,"[High Compatibility] : Phone X/XsMax/Xr ,Phone...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN6LG1-Sy...
3,B07JH1C41D,Wayona Nylon Braided (2 Pack) Lightning Fast U...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/412fvb7k2F...,https://www.amazon.in/Wayona-Braided-WN3LG2-Sy...


##  Content-Based Recommendations (All Product Text Fields)
This method uses all available product text fields for content-based recommendations.

In [8]:
from model_training import get_content_recommendations

try:
    content_recs = get_content_recommendations(df, product_id)
    if isinstance(content_recs, pd.DataFrame) and not content_recs.empty:
        display(content_recs)
    else:
        print("No content-based recommendations found.")
except Exception as e:
    print(f"Error generating content recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JH1CBGW,Wayona Nylon Braided Usb Syncing And Charging ...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/41eHLj-wfG...,https://www.amazon.in/Wayona-Braided-WN3LB2-Sy...
1,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...
2,B07JH1C41D,Wayona Nylon Braided (2 Pack) Lightning Fast U...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/412fvb7k2F...,https://www.amazon.in/Wayona-Braided-WN3LG2-Sy...


## Content-Based with PCA (TF-IDF + PCA)
This method applies PCA to TF-IDF features from product text fields before recommending similar products.

In [9]:
from model_training import get_content_recommendations_pca

try:
    content_pca_recs = get_content_recommendations_pca(df, product_id)
    if isinstance(content_pca_recs, pd.DataFrame) and not content_pca_recs.empty:
        display(content_pca_recs)
    else:
        print("No content-PCA recommendations found.")
except Exception as e:
    print(f"Error generating content PCA recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...
1,B07JH1C41D,Wayona Nylon Braided (2 Pack) Lightning Fast U...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/412fvb7k2F...,https://www.amazon.in/Wayona-Braided-WN3LG2-Sy...
2,B07JH1CBGW,Wayona Nylon Braided Usb Syncing And Charging ...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/41eHLj-wfG...,https://www.amazon.in/Wayona-Braided-WN3LB2-Sy...


## Sentiment-Based Recommendations
This method uses sentiment analysis scores from product reviews to recommend similar products.

In [10]:
from model_training import get_sentiment_recommendations

try:
    sentiment_recs = get_sentiment_recommendations(df, product_id)
    if isinstance(sentiment_recs, pd.DataFrame) and not sentiment_recs.empty:
        display(sentiment_recs)
    else:
        print("No sentiment-based recommendations found.")
except Exception as e:
    print(f"Error generating sentiment recommendations: {e}")

,product_id,product_name,category,rating,discounted_price,img_link
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,4.0,199.0,https://m.media-amazon.com/images/W/WEBP_40237...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,3.9,199.0,https://m.media-amazon.com/images/W/WEBP_40237...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,4.2,329.0,https://m.media-amazon.com/images/I/41V5FtEWPk...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,4.2,154.0,https://m.media-amazon.com/images/W/WEBP_40237...
5,B08Y1TFSP6,pTron Solero TB301 3A Type-C Data and Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,3.9,149.0,https://m.media-amazon.com/images/I/31wOPjcSxl...


## Topic Modeling (LDA)
This method uses topic modeling (LDA) on product reviews to recommend products with similar topics.

In [11]:
from model_training import get_topic_recommendations

try:
    topic_recs = get_topic_recommendations(df, product_id)
    if isinstance(topic_recs, pd.DataFrame) and not topic_recs.empty:
        display(topic_recs)
    else:
        print("No topic-modeling recommendations found.")
except Exception as e:
    print(f"Error generating topic recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...
2,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,449.0,0.0,0.0,4.2,0.0,"[High Compatibility] : Phone X/XsMax/Xr ,Phone...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN6LG1-Sy...
3,B0B4T8RSJ1,pTron Solero T241 2.4A Type-C Data & Charging ...,Computers&Accessories|Accessories&Peripherals|...,99.0,0.0,0.0,3.9,0.0,Solero T241 data and charging cable is equippe...,"AF477BP57JM7Z4JD4PYB2K33R6AQ,AGTDD34Y77OB36JNY...","Placeholder,श्रीPKजी,Gauri salkar,aQeel,Ashish...","R1Q323BB35OP30,RJ0CSQUUWFF9W,R23OB4XMH3S9QD,R1...","The metal pin is losing it's strength,Sahi h b...",It's a good data cable and I recommend it to p...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/pTron-Charging-480Mbps-D...


##  Reviewer Overlap Recommendations
This method recommends products based on overlap in reviewers (users who reviewed both products).

In [12]:
from model_training import get_reviewer_overlap_recommendations

try:
    reviewer_overlap_recs = get_reviewer_overlap_recommendations(df, product_id)
    if isinstance(reviewer_overlap_recs, pd.DataFrame) and not reviewer_overlap_recs.empty:
        display(reviewer_overlap_recs)
    else:
        print("No reviewer-overlap recommendations found.")
except Exception as e:
    print(f"Error generating reviewer-overlap recommendations: {e}")

,product_id,product_name,category,rating,discounted_price,img_link
0,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,4.2,399.0,https://m.media-amazon.com/images/W/WEBP_40237...
1,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,4.2,399.0,https://m.media-amazon.com/images/W/WEBP_40237...
2,B07JH1C41D,Wayona Nylon Braided (2 Pack) Lightning Fast U...,Computers&Accessories|Accessories&Peripherals|...,4.2,649.0,https://m.media-amazon.com/images/I/412fvb7k2F...
3,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,4.2,449.0,https://m.media-amazon.com/images/W/WEBP_40237...
4,B07JH1CBGW,Wayona Nylon Braided Usb Syncing And Charging ...,Computers&Accessories|Accessories&Peripherals|...,4.2,649.0,https://m.media-amazon.com/images/I/41eHLj-wfG...


##  Weighted Hybrid Recommendations
This method combines multiple recommendation sources using weighted scores for a hybrid approach.

In [13]:
from model_training import get_weighted_hybrid_recommendations

try:
    weighted_hybrid_recs = get_weighted_hybrid_recommendations(df, product_id)
    if isinstance(weighted_hybrid_recs, pd.DataFrame) and not weighted_hybrid_recs.empty:
        display(weighted_hybrid_recs)
    else:
        print("No weighted-hybrid recommendations found.")
except Exception as e:
    print(f"Error generating weighted hybrid recommendations: {e}")

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link,hybrid_score
0,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LB1-Sy...,5.4
1,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,449.0,0.0,0.0,4.2,0.0,"[High Compatibility] : Phone X/XsMax/Xr ,Phone...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN6LG1-Sy...,3.4
2,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,399.0,0.0,0.0,4.2,0.0,[High Compatibility] : Compatible For iPhone X...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-Syncing-C...,3.4
3,B07JH1C41D,Wayona Nylon Braided (2 Pack) Lightning Fast U...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/412fvb7k2F...,https://www.amazon.in/Wayona-Braided-WN3LG2-Sy...,3.1
4,B07JH1CBGW,Wayona Nylon Braided Usb Syncing And Charging ...,Computers&Accessories|Accessories&Peripherals|...,649.0,0.0,0.0,4.2,0.0,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/I/41eHLj-wfG...,https://www.amazon.in/Wayona-Braided-WN3LB2-Sy...,2.5


START TRAINING - At this point we have different ways to recommend a product to the user. 

We will test all the product recommendation models to each KPI we set 

In [14]:
import time
def generate_category_ground_truth(df):
    """
    Creates a mapping of product_id -> list of other product_ids 
    in the same exact category.
    """
    ground_truth = {}
    # Group by the category string
    category_groups = df.groupby('category')['product_id'].apply(list)
    
    for category, products in category_groups.items():
        for i, target_id in enumerate(products):
            # All other products in this category are 'relevant'
            relevant = [p for j, p in enumerate(products) if i != j]
            if relevant:
                ground_truth[target_id] = relevant
                
    return ground_truth


def evaluate_recommender(test_data, get_rec_func, K=5):
    """
    Updated: No longer passes 'df' to the recommendation function.
    """
    precisions, reciprocal_ranks, ndcgs, latencies = [], [], [], []

    for target_id, relevant_ids in test_data.items():
        # Measure latency
        start = time.time()
        # FIX: Only pass product_id and N
        recs = get_rec_func(target_id, N=K) 
        latency = time.time() - start
        latencies.append(latency)

        if hasattr(recs, 'product_id'):
            recommended_ids = recs['product_id'].tolist()
        elif isinstance(recs, list):
            recommended_ids = [r['product_id'] for r in recs if 'product_id' in r]
        else:
            recommended_ids = []

        # Precision@K
        hits = len(set(recommended_ids) & set(relevant_ids))
        precisions.append(hits / K)

        # MRR (Mean Reciprocal Rank)
        rank = 0
        for i, rid in enumerate(recommended_ids):
            if rid in relevant_ids:
                rank = i + 1
                break
        reciprocal_ranks.append(1 / rank if rank > 0 else 0)

        # NDCG calculation
        y_true = np.array([[1 if rid in relevant_ids else 0 for rid in recommended_ids]])
        y_score = np.array([[K - i for i in range(len(recommended_ids))]])
        try:
            if np.sum(y_true) > 0:
                ndcgs.append(ndcg_score(y_true, y_score, k=K))
            else:
                ndcgs.append(0.0)
        except:
            ndcgs.append(0.0)

    return {
        f"Precision@{K}": np.mean(precisions),
        "MRR": np.mean(reciprocal_ranks),
        f"NDCG@{K}": np.mean(ndcgs),
        "Latency (s)": np.mean(latencies)
    }


# Ensure KNN features are ready
df, X_knn = prepare_features_for_knn(df)

test_set = generate_category_ground_truth(df)
# Updated test_methods to match the (pid, N) signature exactly
test_methods = {
    "Basic Cosine": lambda pid, N: get_recommendations(df, pid, N=N),
    "PCA Features": lambda pid, N: get_recommendations_with_pca(df, pid, N=N),
    "Content (TF-IDF)": lambda pid, N: get_content_recommendations(df, pid, N=N),
    "Content + PCA": lambda pid, N: get_content_recommendations_pca(df, pid, N=N),
    "Review Text": lambda pid, N: get_review_recommendations(df, pid, N=N),
    "Sentiment": lambda pid, N: get_sentiment_recommendations(df, pid, N=N),
    "Topic Modeling": lambda pid, N: get_topic_recommendations(df, pid, N=N),
    "Reviewer Overlap": lambda pid, N: get_reviewer_overlap_recommendations(df, pid, N=N),
    "SVD (Collab)": lambda pid, N: svd_product_recommendations(df, pid, n=N),
    "KNN (Numeric)": lambda pid, N: get_knn_recommendations(df, X_knn, df[df['product_id']==pid].index[0], n=N)
   # "OPTIMIZED HYBRID": lambda pid, N: get_weighted_hybrid_recommendations(pid, N=N, weights=optimal_weights)
}

# Run the evaluation
results = []
test_set_filtered = {k: v for k, v in test_set.items() if len(v) > 1}
eval_sample = dict(list(test_set_filtered.items())[:10])
print(f"Sampled {len(eval_sample)} queries for evaluation.")

#eval_sample = dict(list(hybrid_truth.items())[:30]) 

for name, func in test_methods.items():
    print(f"Evaluating {name}...", end=" ")
    try:
        metrics = evaluate_recommender(eval_sample, func, K=5)
        metrics['Model'] = name
        results.append(metrics)
        print("Done.")
    except Exception as e:
        print(f"Failed: {e}")

# Display results
kpi_df = pd.DataFrame(results)
kpi_df = kpi_df[['Model', 'NDCG@5', 'MRR', 'Precision@5', 'Latency (s)']]
display(kpi_df.sort_values(by='NDCG@5', ascending=False))

Sampled 10 queries for evaluation.
Evaluating Basic Cosine... Done.
Evaluating PCA Features... Done.
Evaluating Content (TF-IDF)... Done.
Evaluating Content + PCA... Done.
Evaluating Review Text... Done.
Evaluating Sentiment... Done.
Evaluating Topic Modeling... Done.
Evaluating Reviewer Overlap... Done.
Evaluating SVD (Collab)... Done.
Evaluating KNN (Numeric)... Done.


,Model,NDCG@5,MRR,Precision@5,Latency (s)
5,Sentiment,0.960408,0.950000,0.90,0.001116
0,Basic Cosine,0.706022,0.550000,0.38,0.002272
3,Content + PCA,0.703025,0.550000,0.36,0.001911
2,Content (TF-IDF),0.699085,0.550000,0.34,0.001732
8,SVD (Collab),0.690497,0.650000,0.34,0.556520
1,PCA Features,0.683776,0.650000,0.42,0.002047
4,Review Text,0.666886,0.600000,0.38,0.001722
6,Topic Modeling,0.647691,0.583333,0.30,0.001715
7,Reviewer Overlap,0.500000,0.600000,0.30,0.001690
9,KNN (Numeric),0.488198,0.375000,0.26,0.001500


Interpretation

Our new goal is to combine these models in achieving optima NDCG score, this will be a hybrid type of recommendation engine, and we will use grid search to find the 'sweet spot' 

In [15]:
import numpy as np
import pandas as pd
import random
from model_training import get_weighted_hybrid_recommendations

def randomized_weight_search(df, test_data, n_iter=50, K=5):
    """
    Randomly samples weight combinations to find the best configuration.
    """
    model_keys = [
        'basic_cosine', 'pca_features', 'content_tfidf', 'content_pca', 
        'review_text', 'sentiment', 'topic_lda', 'reviewer_overlap', 
        'knn_numeric', 'svd_collaborative'
    ]
    
    results = []
    best_score = -1
    best_weights = None

    print(f"Starting randomized search for {n_iter} iterations...")

    for i in range(n_iter):
        # 1. Generate random weights that sum to 1.0
        raw_weights = np.random.dirichlet(np.ones(len(model_keys)), size=1)[0]
        current_weights = dict(zip(model_keys, raw_weights))

        # 2. Updated wrapper signature to match evaluate_recommender (pid, N)
        def hybrid_func(pid, N):
            # Pass the global df to the hybrid function as required by model_training.py
            return get_weighted_hybrid_recommendations(df, pid, N=N, weights=current_weights)

        # 3. Evaluate using the category-based truth
        scores = evaluate_recommender(test_data, hybrid_func, K=K)
        score = scores[f'NDCG@{K}']
        
        results.append({'weights': current_weights, 'ndcg': score})

        if score > best_score:
            best_score = score
            best_weights = current_weights
            print(f"Iteration {i}: New Best NDCG@{K} = {best_score:.4f}")

    return best_weights, best_score

# --- Run the Optimization ---
category_truth = generate_category_ground_truth(df)
sample_truth = dict(list(category_truth.items())[:20]) 

optimal_weights, top_score = randomized_weight_search(df, sample_truth, n_iter=30)

print("\n--- OPTIMAL HYBRID WEIGHTS ---")
for k, v in optimal_weights.items():
    print(f"{k:20}: {v:.4f}")

Starting randomized search for 30 iterations...
Iteration 0: New Best NDCG@5 = 0.6600
Iteration 2: New Best NDCG@5 = 0.6985
Iteration 3: New Best NDCG@5 = 0.9615
Iteration 5: New Best NDCG@5 = 1.0000

--- OPTIMAL HYBRID WEIGHTS ---
basic_cosine        : 0.0293
pca_features        : 0.0883
content_tfidf       : 0.2967
content_pca         : 0.1711
review_text         : 0.0779
sentiment           : 0.0935
topic_lda           : 0.0709
reviewer_overlap    : 0.1381
knn_numeric         : 0.0085
svd_collaborative   : 0.0257


In [16]:
import time
import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score

# 1. Setup Ground Truth (Filtered for rich data)
test_set = generate_category_ground_truth(df)
test_set_filtered = {k: v for k, v in test_set.items() if len(v) > 1}
# Expanding sample size to 30 for more stable KPIs
eval_sample = dict(list(test_set_filtered.items())[:30]) 

print(f"Evaluating Optimized Hybrid on {len(eval_sample)} queries...")

# 2. Define the Optimized Hybrid Wrapper
# Uses your best weights from the randomized search
best_hybrid_weights = optimal_weights  # From previous optimization step

# The lambda ensures the evaluator only needs to pass (pid, N)
hybrid_func = lambda pid, N: get_weighted_hybrid_recommendations(df, pid, N=N, weights=best_hybrid_weights)

# 3. Run Single Evaluation
try:
    start_wall = time.time()
    metrics = evaluate_recommender(eval_sample, hybrid_func, K=5)
    total_time = time.time() - start_wall
    
    metrics['Model'] = "OPTIMIZED HYBRID"
    
    # 4. Display results in a clean table
    kpi_df = pd.DataFrame([metrics])
    cols = ['Model', 'NDCG@5', 'MRR', 'Precision@5', 'Latency (s)']
    display(kpi_df[cols])
    
    print(f"\nTotal Wall Time for 30 samples: {total_time:.2f} seconds")

except Exception as e:
    print(f"Evaluation failed: {e}")

Evaluating Optimized Hybrid on 30 queries...


,Model,NDCG@5,MRR,Precision@5,Latency (s)
0,OPTIMIZED HYBRID,1.0,1.0,1.0,0.754544



Total Wall Time for 30 samples: 22.65 seconds


it met our target KPIs but our evaluator is category based let's try a hybrid style

In [17]:
def generate_hybrid_ground_truth(df, weight_category=0.5, weight_behavior=0.5):
    """
    Creates a mapping of product_id -> list of relevant product_ids.
    Relevance is a weighted score of Category Match and Reviewer Overlap.
    """
    ground_truth = {}
    
    # Pre-group by category for faster lookup
    cat_groups = df.groupby('category')['product_id'].apply(list).to_dict()
    
    # Iterate through all products in the dataframe
    for target_id in df['product_id'].unique():
        scores = {}
        
        # --- Signal 1: Category Match ---
        target_cat = df[df['product_id'] == target_id]['category'].iloc[0]
        category_peers = cat_groups.get(target_cat, [])
        for peer in category_peers:
            if peer != target_id:
                scores[peer] = scores.get(peer, 0) + weight_category
                
        # --- Signal 2: Behavioral Reviewer Overlap ---
        # Find users who reviewed the target product
        target_users = set(df[df['product_id'] == target_id]['user_id'].iloc[0].split(','))
        
        # Identify other products these users have reviewed
        # Using the same logic as your get_reviewer_overlap_recommendations
        overlap_products = df[df['user_id'].str.contains('|'.join(target_users)) & (df['product_id'] != target_id)]
        
        if not overlap_products.empty:
            counts = overlap_products['product_id'].value_counts()
            max_count = counts.max()
            for pid, count in counts.items():
                # Normalize overlap count and apply weight
                norm_score = (count / max_count) * weight_behavior
                scores[pid] = scores.get(pid, 0) + norm_score

        # Store top relevant products (sorted by score)
        if scores:
            sorted_recs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            ground_truth[target_id] = [p[0] for p in sorted_recs[:50]]
                
    return ground_truth

In [18]:
# 1. Generate the truth
hybrid_truth = generate_hybrid_ground_truth(df)

# 2. Sample 20 products for the search
sample_truth = dict(list(hybrid_truth.items())[:20]) 

# 3. Run the optimized search
optimal_weights, top_score = randomized_weight_search(df, sample_truth, n_iter=30)

print("\n--- OPTIMAL HYBRID WEIGHTS ---")
for k, v in optimal_weights.items():
    print(f"{k:20}: {v:.4f}")

Starting randomized search for 30 iterations...
Iteration 0: New Best NDCG@5 = 0.7483
Iteration 2: New Best NDCG@5 = 0.8104
Iteration 8: New Best NDCG@5 = 0.8922

--- OPTIMAL HYBRID WEIGHTS ---
basic_cosine        : 0.0008
pca_features        : 0.2657
content_tfidf       : 0.2715
content_pca         : 0.0623
review_text         : 0.0838
sentiment           : 0.0386
topic_lda           : 0.0405
reviewer_overlap    : 0.2058
knn_numeric         : 0.0091
svd_collaborative   : 0.0218


let's see first the KPI metrics from the model based on the new hybrid ground truth

In [19]:
import time
def evaluate_recommender(test_data, get_rec_func, K=5):
    """
    Updated: No longer passes 'df' to the recommendation function.
    """
    precisions, reciprocal_ranks, ndcgs, latencies = [], [], [], []

    for target_id, relevant_ids in test_data.items():
        # Measure latency
        start = time.time()
        # FIX: Only pass product_id and N
        recs = get_rec_func(target_id, N=K) 
        latency = time.time() - start
        latencies.append(latency)

        if hasattr(recs, 'product_id'):
            recommended_ids = recs['product_id'].tolist()
        elif isinstance(recs, list):
            recommended_ids = [r['product_id'] for r in recs if 'product_id' in r]
        else:
            recommended_ids = []

        # Precision@K
        hits = len(set(recommended_ids) & set(relevant_ids))
        precisions.append(hits / K)

        # MRR (Mean Reciprocal Rank)
        rank = 0
        for i, rid in enumerate(recommended_ids):
            if rid in relevant_ids:
                rank = i + 1
                break
        reciprocal_ranks.append(1 / rank if rank > 0 else 0)

        # NDCG calculation
        y_true = np.array([[1 if rid in relevant_ids else 0 for rid in recommended_ids]])
        y_score = np.array([[K - i for i in range(len(recommended_ids))]])
        try:
            if np.sum(y_true) > 0:
                ndcgs.append(ndcg_score(y_true, y_score, k=K))
            else:
                ndcgs.append(0.0)
        except:
            ndcgs.append(0.0)

    return {
        f"Precision@{K}": np.mean(precisions),
        "MRR": np.mean(reciprocal_ranks),
        f"NDCG@{K}": np.mean(ndcgs),
        "Latency (s)": np.mean(latencies)
    }


# Ensure KNN features are ready
df, X_knn = prepare_features_for_knn(df)


hybrid_truth = generate_hybrid_ground_truth(df)
# Updated test_methods to match the (pid, N) signature exactly
test_methods = {
    "Basic Cosine": lambda pid, N: get_recommendations(df, pid, N=N),
    "PCA Features": lambda pid, N: get_recommendations_with_pca(df, pid, N=N),
    "Content (TF-IDF)": lambda pid, N: get_content_recommendations(df, pid, N=N),
    "Content + PCA": lambda pid, N: get_content_recommendations_pca(df, pid, N=N),
    "Review Text": lambda pid, N: get_review_recommendations(df, pid, N=N),
    "Sentiment": lambda pid, N: get_sentiment_recommendations(df, pid, N=N),
    "Topic Modeling": lambda pid, N: get_topic_recommendations(df, pid, N=N),
    "Reviewer Overlap": lambda pid, N: get_reviewer_overlap_recommendations(df, pid, N=N),
    "SVD (Collab)": lambda pid, N: svd_product_recommendations(df, pid, n=N),
    "KNN (Numeric)": lambda pid, N: get_knn_recommendations(df, X_knn, df[df['product_id']==pid].index[0], n=N)
   # "OPTIMIZED HYBRID": lambda pid, N: get_weighted_hybrid_recommendations(pid, N=N, weights=optimal_weights)
}

# Run the evaluation
results = []
test_set_filtered = {k: v for k, v in test_set.items() if len(v) > 1}
eval_sample = dict(list(test_set_filtered.items())[:10])
print(f"Sampled {len(eval_sample)} queries for evaluation.")

#eval_sample = dict(list(hybrid_truth.items())[:30]) 

for name, func in test_methods.items():
    print(f"Evaluating {name}...", end=" ")
    try:
        metrics = evaluate_recommender(eval_sample, func, K=5)
        metrics['Model'] = name
        results.append(metrics)
        print("Done.")
    except Exception as e:
        print(f"Failed: {e}")

# Display results
kpi_df = pd.DataFrame(results)
kpi_df = kpi_df[['Model', 'NDCG@5', 'MRR', 'Precision@5', 'Latency (s)']]
display(kpi_df.sort_values(by='NDCG@5', ascending=False))

Sampled 10 queries for evaluation.
Evaluating Basic Cosine... Done.
Evaluating PCA Features... Done.
Evaluating Content (TF-IDF)... Done.
Evaluating Content + PCA... Done.
Evaluating Review Text... Done.
Evaluating Sentiment... Done.
Evaluating Topic Modeling... Done.
Evaluating Reviewer Overlap... Done.
Evaluating SVD (Collab)... Done.
Evaluating KNN (Numeric)... Done.


,Model,NDCG@5,MRR,Precision@5,Latency (s)
5,Sentiment,0.960408,0.950000,0.90,0.000761
0,Basic Cosine,0.706022,0.550000,0.38,0.001603
3,Content + PCA,0.703025,0.550000,0.36,0.001489
2,Content (TF-IDF),0.699085,0.550000,0.34,0.001491
1,PCA Features,0.683776,0.650000,0.42,0.001494
4,Review Text,0.666886,0.600000,0.38,0.001486
6,Topic Modeling,0.647691,0.583333,0.30,0.001561
7,Reviewer Overlap,0.500000,0.600000,0.30,0.001420
9,KNN (Numeric),0.488198,0.375000,0.26,0.001627
8,SVD (Collab),0.355065,0.250000,0.26,0.463711


In [20]:
import time
import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score

# 1. Setup Ground Truth (Filtered for rich data)
test_set = generate_hybrid_ground_truth(df)
test_set_filtered = {k: v for k, v in test_set.items() if len(v) > 1}
# Expanding sample size to 30 for more stable KPIs
eval_sample = dict(list(test_set_filtered.items())[:30]) 

print(f"Evaluating Optimized Hybrid on {len(eval_sample)} queries...")

# 2. Define the Optimized Hybrid Wrapper
# Uses your best weights from the randomized search
best_hybrid_weights = optimal_weights  # From previous optimization step

# The lambda ensures the evaluator only needs to pass (pid, N)
hybrid_func = lambda pid, N: get_weighted_hybrid_recommendations(df, pid, N=N, weights=optimal_weights)

# 3. Run Single Evaluation
try:
    start_wall = time.time()
    metrics = evaluate_recommender(eval_sample, hybrid_func, K=5)
    total_time = time.time() - start_wall
    
    metrics['Model'] = "OPTIMIZED HYBRID"
    
    # 4. Display results in a clean table
    kpi_df = pd.DataFrame([metrics])
    cols = ['Model', 'NDCG@5', 'MRR', 'Precision@5', 'Latency (s)']
    display(kpi_df[cols])
    
    print(f"\nTotal Wall Time for 30 samples: {total_time:.2f} seconds")

except Exception as e:
    print(f"Evaluation failed: {e}")

Evaluating Optimized Hybrid on 30 queries...


,Model,NDCG@5,MRR,Precision@5,Latency (s)
0,OPTIMIZED HYBRID,0.905077,0.902778,0.62,0.5805



Total Wall Time for 30 samples: 17.43 seconds
